# **The Data API**
Seluruh Data API berkisar pada konsep _dataset_, yang merepresentasikan urutan item data. _Dataset_ dapat dibuat dari _tensor_ menggunakan `tf.data.Dataset.from_tensor_slices()`.

```python
x = tf.range(10) # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(x)
# dataset sekarang berisi item: tensor 0, 1, 2, ..., 9
```

Item _dataset_ dapat diulang menggunakan _loop_ `for`:
```python
for item in dataset:
    print(item)
```

**Chaining Transformations**
Setelah memiliki _dataset_, berbagai transformasi dapat diterapkan dengan memanggil metode transformasinya. Setiap metode mengembalikan _dataset_ baru, sehingga transformasi dapat dirantai.

Contoh: `repeat(3)` dan `batch(7)`.
```python
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)
```
* `repeat()`: Mengembalikan _dataset_ baru yang akan mengulang item dari _dataset_ asli sebanyak tiga kali. Ini tidak menyalin semua data ke dalam memori tiga kali. Jika dipanggil tanpa argumen, _dataset_ akan diulang selamanya.
* `batch()`: Mengelompokkan item dari _dataset_ sebelumnya dalam _batch_ tujuh item. _Batch_ terakhir mungkin memiliki ukuran yang berbeda, tetapi dapat dihilangkan dengan `drop_remainder=True`.
Metode _dataset_ tidak memodifikasi _dataset_ asli; mereka membuat yang baru.

**Transformasi Item**
* `map()`: Menerapkan transformasi ke setiap item. Misalnya, mendobel setiap item. Ini digunakan untuk _preprocessing_ data. Untuk mempercepat, argumen `num_parallel_calls` dapat diatur untuk _multithreading_. Fungsi yang diteruskan ke `map()` harus dapat diubah menjadi TF Function.
    ```python
    dataset = dataset.map(lambda x: x * 2) # Items: [0,2,4,6,8,10,12]
    ```
* `apply()`: Menerapkan transformasi ke seluruh _dataset_. Contohnya adalah `tf.data.experimental.unbatch()`.
* `filter()`: Menyaring _dataset_ berdasarkan suatu kondisi.
* `take()`: Mengambil beberapa item dari _dataset_.

**Shuffling Data**
_Gradient Descent_ bekerja paling baik ketika _instance_ dalam _training set_ bersifat independen dan terdistribusi secara identik. Mengocok _instance_ dapat dilakukan dengan metode `shuffle()`.
* `shuffle()`: Mengisi _buffer_ dengan item pertama dari _dataset_ sumber. Ketika item diminta, ia akan menarik satu secara acak dari _buffer_ dan menggantinya dengan yang baru dari _dataset_ sumber. Ukuran _buffer_ harus cukup besar agar _shuffling_ efektif.
    ```python
    dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times
    dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
    for item in dataset:
        print(item)
    ```
    Jika `repeat()` dipanggil pada _dataset_ yang di-_shuffle_, secara _default_ akan menghasilkan urutan baru di setiap iterasi. Ini dapat diubah dengan `reshuffle_each_iteration=False`.

Untuk _dataset_ besar yang tidak muat di memori, _shuffling_ sumber data itu sendiri (misalnya, menggunakan perintah `shuf` di Linux) dapat meningkatkan _shuffling_. Pendekatan umum adalah membagi data sumber menjadi beberapa _file_, kemudian membacanya dalam urutan acak selama pelatihan. Untuk menghindari _instance_ dalam _file_ yang sama berakhir berdekatan, beberapa _file_ dapat dipilih secara acak dan dibaca secara bersamaan, dengan menginterleave catatan mereka. Kemudian _shuffling buffer_ dapat ditambahkan di atasnya.

**Interleaving lines from multiple files**
_Data API_ memungkinkan hal ini hanya dengan beberapa baris kode.
1.  Buat _dataset_ berisi jalur _file_ menggunakan `tf.data.Dataset.list_files()`. Secara _default_, `list_files()` mengocok jalur _file_.
    ```python
    filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)
    ```
2.  Gunakan metode `interleave()` untuk membaca dari beberapa _file_ sekaligus dan menginterleave barisnya. _Callback_ fungsi (lambda dalam contoh) akan membuat _dataset_ baru untuk setiap jalur _file_ (misalnya, `tf.data.TextLineDataset`).
    ```python
    n_readers = 5
    dataset = filepath_dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers)
    ```
    Secara _default_, `interleave()` tidak menggunakan paralelisme. Untuk membaca _file_ secara paralel, atur argumen `num_parallel_calls` atau `tf.data.experimental.AUTOTUNE`.

**Preprocessing the Data**
Data perlu di-_parse_ dan di-_scale_. Contoh fungsi _preprocess_ untuk data CSV:
```python
X_mean, X_std = [...] # mean and scale of each feature in the training set
n_inputs = 8

def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y
```
* `tf.io.decode_csv()`: _Parse_ baris CSV. Argumen kedua adalah _array_ nilai _default_ untuk setiap kolom, yang juga memberi tahu TensorFlow jumlah kolom dan tipenya.
* `tf.stack()`: Menggabungkan _scalar tensors_ menjadi _1D tensor arrays_.
* Fitur _input_ diskalakan dengan mengurangi _mean_ dan membagi dengan _standard deviation_.

**Putting Everything Together**
Fungsi _helper_ `csv_reader_dataset` dapat dibuat untuk memuat, memproses, mengocok, mengulang, dan _batching_ data secara efisien.
```python
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=1000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)
```

**Prefetching**
Memanggil `prefetch(1)` di akhir _pipeline_ akan membuat _dataset_ selalu selangkah lebih maju, menyiapkan _batch_ berikutnya secara paralel saat _batch_ saat ini sedang diproses oleh algoritma pelatihan. Ini dapat meningkatkan kinerja secara dramatis. Dengan _multithreaded loading_ dan _preprocessing_ (mengatur `num_parallel_calls` di `interleave()` dan `map()`), GPU dapat dimanfaatkan hampir 100%.

Jika _dataset_ cukup kecil untuk muat di memori, `cache()` dapat mempercepat pelatihan secara signifikan dengan menyimpan konten ke RAM. Ini harus dilakukan setelah memuat dan memproses data, tetapi sebelum _shuffling_, _repeating_, _batching_, dan _prefetching_.

**Using the Dataset with tf.keras**
_Dataset_ yang dibuat dapat langsung diteruskan ke metode `fit()`, `evaluate()`, dan `predict()` pada model Keras.
```python
train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

model = keras.models.Sequential([...])
model.compile([...])
model.fit(train_set, epochs=10, validation_data=valid_set)
```
Keras akan menangani pengulangan _training dataset_.

**The TFRecord Format**
TFRecord adalah format biner pilihan TensorFlow untuk menyimpan data dalam jumlah besar dan membacanya secara efisien. Ini hanya berisi urutan catatan biner dengan ukuran bervariasi.
* Membuat _file_ TFRecord menggunakan `tf.io.TFRecordWriter`.
    ```python
    with tf.io.TFRecordWriter("my_data.tfrecord") as f:
        f.write(b"This is the first record")
        f.write(b"And this is the second record")
    ```
* Membaca _file_ TFRecord menggunakan `tf.data.TFRecordDataset`.
    ```python
    filepaths = ["my_data.tfrecord"]
    dataset = tf.data.TFRecordDataset(filepaths)
    for item in dataset:
        print(item)
    ```
    Secara _default_, `TFRecordDataset` akan membaca _file_ satu per satu, tetapi dapat membaca beberapa _file_ secara paralel dan menginterleave catatannya dengan mengatur `num_parallel_reads`.

**Compressed TFRecord Files**
TFRecord _file_ dapat dikompresi, terutama jika perlu dimuat melalui koneksi jaringan.
* Membuat _file_ TFRecord terkompresi dengan mengatur argumen `options`.
    ```python
    options = tf.io.TFRecordOptions(compression_type="GZIP")
    with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
        # ...
    ```
* Saat membaca _file_ TFRecord terkompresi, jenis kompresi perlu ditentukan.
    ```python
    dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                      compression_type="GZIP")
    ```

**A Brief Introduction to Protocol Buffers**
_File_ TFRecord biasanya berisi _serialized protocol buffers_ (protobufs). Ini adalah format biner yang portabel, _extensible_, dan efisien yang dikembangkan oleh Google. Protobufs didefinisikan menggunakan bahasa sederhana. TensorFlow menyertakan definisi protobuf khusus yang menyediakan operasi _parsing_.

**TensorFlow Protobufs**
Protobuf utama yang biasanya digunakan dalam _file_ TFRecord adalah _Example protobuf_, yang merepresentasikan satu _instance_ dalam _dataset_. Ini berisi daftar fitur bernama, di mana setiap fitur dapat berupa daftar _byte strings_, _floats_, atau _integers_.
```python
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com",
                                                        b"c@d.com"]))
        }))
```
Objek _Example_ dapat diserialisasi menggunakan `SerializeToString()` dan kemudian ditulis ke _file_ TFRecord.

**Loading and Parsing Examples**
Untuk memuat _serialized Example protobufs_, gunakan `tf.data.TFRecordDataset` dan _parse_ setiap _Example_ menggunakan `tf.io.parse_single_example()`. Ini adalah operasi TensorFlow dan dapat dimasukkan dalam TF Function. Ini membutuhkan data _serialized_ dan deskripsi setiap fitur. Deskripsi adalah _dictionary_ yang memetakan nama fitur ke _descriptor_ `tf.io.FixedLenFeature` (untuk fitur panjang tetap) atau `tf.io.VarLenFeature` (untuk fitur panjang bervariasi).
```python
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}

for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
    parsed_example = tf.io.parse_single_example(serialized_example,
                                                feature_description)
```
Fitur panjang tetap di-_parse_ sebagai _regular tensors_, sedangkan fitur panjang variabel di-_parse_ sebagai _sparse tensors_. _Sparse tensor_ dapat diubah menjadi _dense tensor_ menggunakan `tf.sparse.to_dense()` atau nilai-nilainya dapat diakses langsung.

`BytesList` dapat berisi data biner apa pun. Gambar dapat di-_encode_ menggunakan `tf.io.encode_jpeg()` dan kemudian di-_decode_ menggunakan `tf.io.decode_jpeg()` atau `tf.io.decode_image()`. _Tensor_ dapat diserialisasi dengan `tf.io.serialize_tensor()` dan di-_parse_ dengan `tf.io.parse_tensor()`.

Untuk _parse_ _Example_ secara _batch_, gunakan `tf.io.parse_example()`.
```python
dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"]).batch(10)
for serialized_examples in dataset:
    parsed_examples = tf.io.parse_example(serialized_examples,
                                          feature_description)
```

**Handling Lists of Lists Using the SequenceExample Protobuf**
_SequenceExample protobuf_ dirancang untuk kasus penggunaan dengan daftar-daftar (misalnya, dokumen sebagai daftar kalimat, setiap kalimat sebagai daftar kata).
* _SequenceExample_ berisi objek `Features` untuk data kontekstual dan objek `FeatureLists` yang berisi satu atau lebih objek `FeatureList` bernama. Setiap `FeatureList` berisi daftar objek `Feature`.
* _SequenceExample_ di-_parse_ menggunakan `tf.io.parse_single_sequence_example()` untuk _single example_ atau `tf.io.parse_sequence_example()` untuk _batch_. Fungsi-fungsi ini mengembalikan _tuple_ berisi fitur konteks (_dictionary_) dan daftar fitur (_dictionary_). Jika daftar fitur berisi urutan dengan ukuran bervariasi, dapat diubah menjadi _ragged tensors_ menggunakan `tf.RaggedTensor.from_sparse()`.

**Preprocessing the Input Features**
Mempersiapkan data untuk _neural network_ melibatkan konversi semua fitur menjadi fitur numerik, umumnya menormalisasikannya. Fitur kategorikal atau teks perlu dikonversi ke angka. Ini dapat dilakukan sebelumnya saat menyiapkan _file_ data, dalam _tf.data pipeline_, atau dalam _preprocessing layers_ di dalam model.

**Standardization Layer (Custom)**
Contoh _standardization layer_ menggunakan _Lambda layer_ atau _custom layer_.
```python
# Menggunakan Lambda layer:
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
    keras.layers.Lambda(lambda inputs: (inputs - means) / (stds + eps)),
    # ... other layers
])

# Custom layer:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

# Penggunaan custom layer:
std_layer = Standardization()
std_layer.adapt(data_sample)
model = keras.Sequential()
model.add(std_layer)
# ... create the rest of the model
```
_Keras_ kemungkinan akan menyediakan _keras.layers.Normalization_ layer yang bekerja serupa.

**Encoding Categorical Features Using One-Hot Vectors**
Untuk fitur kategorikal dengan sedikit kategori, _one-hot encoding_ dapat digunakan. Ini melibatkan pemetaan setiap kategori ke indeksnya menggunakan _lookup table_.
```python
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
```
_Out-of-vocabulary (oov)_ _buckets_ digunakan untuk kategori yang tidak ada dalam _vocabulary_. `tf.one_hot()` mengonversi indeks menjadi _one-hot vectors_. _Keras_ kemungkinan akan menyertakan _keras.layers.TextVectorization_ yang dapat melakukan hal ini.

**Encoding Categorical Features Using Embeddings**
_Embedding_ adalah _dense vector_ yang dapat dilatih yang merepresentasikan sebuah kategori. _Embeddings_ diinisialisasi secara acak dan akan membaik selama pelatihan.
* Ukuran _embedding_ adalah _hyperparameter_.
* _Embedding matrix_ dibuat dengan satu baris per kategori (dan _oov bucket_) dan satu kolom per dimensi _embedding_.
    ```python
    embedding_dim = 2
    embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
    embedding_matrix = tf.Variable(embed_init)
    ```
* `tf.nn.embedding_lookup()`: Mencari baris dalam _embedding matrix_ pada indeks yang diberikan.
    ```python
    tf.nn.embedding_lookup(embedding_matrix, cat_indices)
    ```
* _Keras_ menyediakan _keras.layers.Embedding_ layer yang menangani _embedding matrix_.
    ```python
    embedding = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets,
                                        output_dim=embedding_dim)
    embedding(cat_indices)
    ```

Model Keras dapat mengombinasikan _regular inputs_ dengan _categorical inputs_ yang di-_encode_ menggunakan _embeddings_.
```python
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[regular_inputs, categories],
                           outputs=[outputs])
```

**Keras Preprocessing Layers**
Tim TensorFlow sedang mengembangkan serangkaian _standard Keras preprocessing layers_.
* `keras.layers.Normalization`: Melakukan standardisasi fitur.
* `keras.layers.TextVectorization`: Meng-_encode_ setiap kata menjadi indeksnya dalam _vocabulary_.
* `keras.layers.Discretization`: Memotong data kontinu menjadi beberapa _bin_ dan meng-_encode_ setiap _bin_ sebagai _one-hot vector_.
    ```python
    normalization = keras.layers.Normalization()
    discretization = keras.layers.Discretization([...])
    pipeline = keras.layers.PreprocessingStage([normalization, discretization])
    pipeline.adapt(data_sample)
    ```
_Preprocessing layers_ model akan dibekukan selama pelatihan, sehingga parameternya tidak akan terpengaruh oleh _Gradient Descent_.

`TextVectorization` juga akan memiliki opsi untuk menghasilkan _word-count vectors_ (_bag of words_). _Word counts_ dapat dinormalisasi menggunakan _Term-Frequency Inverse-Document-Frequency (TF-IDF)_.

**TF Transform**
Jika _preprocessing_ mahal secara komputasi, menanganinya sebelum pelatihan dapat memberikan _speedup_ yang signifikan. TF Transform memungkinkan untuk menulis satu fungsi _preprocessing_ yang dapat dijalankan dalam mode _batch_ pada _training set_ lengkap sebelum pelatihan, dan kemudian diekspor ke TF Function dan dimasukkan ke dalam model yang dilatih untuk menangani _preprocessing_ _instance_ baru saat _deployed_. Ini menghindari masalah _training/serving skew_.
```python
import tensorflow_transform as tft

def preprocess(inputs): # inputs a batch of input features
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]
    standardized_age = tft.scale_to_z_score(median_age)
    ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_proximity)
    return {
        "standardized_median_age": standardized_age,
        "ocean_proximity_id": ocean_proximity_id
    }
```
TF Transform dapat menerapkan fungsi `preprocess()` ke seluruh _training set_ menggunakan Apache Beam. Ini juga akan menghitung statistik yang diperlukan (misalnya, _mean_, _standard deviation_, _vocabulary_). Selain itu, TF Transform akan menghasilkan TF Function yang setara yang dapat dicolokkan ke model yang akan _deployed_.

**The TensorFlow Datasets (TFDS) Project**
TFDS mempermudah pengunduhan _common dataset_.
* Instal _tensorflow-datasets_ library.
* Panggil `tfds.load()` untuk mengunduh data dan mengembalikan data sebagai _dictionary_ dari _datasets_.
    ```python
    import tensorflow_datasets as tfds

    dataset = tfds.load(name="mnist")
    mnist_train, mnist_test = dataset["train"], dataset["test"]
    ```
* Dapat menerapkan transformasi apa pun (misalnya, _shuffling_, _batching_, dan _prefetching_).
* Untuk model Keras, setiap item harus berupa _tuple_ berisi fitur dan _label_. Ini dapat diatur dengan `as_supervised=True` saat memanggil `load()`.
    ```python
    dataset = tfds.load(name="mnist", batch_size=32, as_supervised=True)
    mnist_train = dataset["train"].prefetch(1)

    model = keras.models.Sequential([...])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
    model.fit(mnist_train, epochs=5)
    ```